In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import stim
from stimrgs_v1.utils import *
from stimrgs_v1.stabilizer_operations import *
from tqdm import tqdm

## Stabilizer results

In [3]:
first_row = [0,1,2]
second_row = [3,4,5]
third_row = [6,7,8]
fouth_row = [9,10,11]
all_stabilizer_result = []

for i in first_row:
    for j in second_row:
        for k in third_row:
            for l in fouth_row:
                # Create s_1 and replace character at index j with 'Z'
                s_1 = '_' * i + 'X' + '_' * (11 - i) 
                s_1 = s_1[:j] + 'Z' + s_1[j+1:]  # Create a new string with 'Z' at the desired position                

                # Create s_2 and replace character at index k with 'Z'
                s_2 = '_' * j + 'X' + '_' * (11 - j)
                s_2 = s_2[:k] + 'Z' + s_2[k+1:]  # Create a new string with 'Z' at the desired position
                # Create a new string with 'Z' at the desired index i
                s_2 = s_2[:i] + 'Z' + s_2[i+1:]  # Replace character at index i with 'X'

                s_3 = '_' * k + 'X' + '_' * (11 - k)
                s_3 = s_3[:l] + 'Z' + s_3[l+1:]  
                # Create a new string with 'Z' at the desired position
                s_3 = s_3[:j] + 'Z' + s_3[j+1:]  # Replace character at index i with 'X'

                s_4 = '_' * l + 'X' + '_' * (11 - l)
                s_4 = s_4[:k] + 'Z' + s_4[k+1:]  # Create a new string with 'Z' at the desired position

                stabilizer_result = [s_1, s_2, s_3, s_4]
                all_stabilizer_result.append(stabilizer_result)

if len(all_stabilizer_result) != (3**4):
    raise ValueError("The number of stabilizer strings is not correct")

all_stabilizer_result

[['X__Z________', 'Z__X__Z_____', '___Z__X__Z__', '______Z__X__'],
 ['X__Z________', 'Z__X__Z_____', '___Z__X___Z_', '______Z___X_'],
 ['X__Z________', 'Z__X__Z_____', '___Z__X____Z', '______Z____X'],
 ['X__Z________', 'Z__X___Z____', '___Z___X_Z__', '_______Z_X__'],
 ['X__Z________', 'Z__X___Z____', '___Z___X__Z_', '_______Z__X_'],
 ['X__Z________', 'Z__X___Z____', '___Z___X___Z', '_______Z___X'],
 ['X__Z________', 'Z__X____Z___', '___Z____XZ__', '________ZX__'],
 ['X__Z________', 'Z__X____Z___', '___Z____X_Z_', '________Z_X_'],
 ['X__Z________', 'Z__X____Z___', '___Z____X__Z', '________Z__X'],
 ['X___Z_______', 'Z___X_Z_____', '____Z_X__Z__', '______Z__X__'],
 ['X___Z_______', 'Z___X_Z_____', '____Z_X___Z_', '______Z___X_'],
 ['X___Z_______', 'Z___X_Z_____', '____Z_X____Z', '______Z____X'],
 ['X___Z_______', 'Z___X__Z____', '____Z__X_Z__', '_______Z_X__'],
 ['X___Z_______', 'Z___X__Z____', '____Z__X__Z_', '_______Z__X_'],
 ['X___Z_______', 'Z___X__Z____', '____Z__X___Z', '_______Z___

In [4]:
expected_num_bells = 2

for num_measurements in range(1, 11):
    print(f'Measure: {num_measurements} nodes')
    combinations = generate_combinations_with_adjustable_replacement(
        nodes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 
        max_length=num_measurements, # You have to change this parameter, if you want to modify number of node operation
        max_self_combination=1
    )    

    for c in combinations:

        # Restart parameters before operate new measurement set
        found: bool = False
        bell_found_list = []
        s = stim_rgs_3arms_2bells()
        q = [
        'X___ZZ______', # node 0
        '_X_Z_Z______', # node 1
        '__XZZ_______', # node 2
        '_ZZX__Z_____', # node 3
        'Z_Z_X__Z____', # node 4
        'ZZ___X__Z___', # node 5
        '___Z__X___ZZ', # node 6
        '____Z__X_Z_Z', # node 7
        '_____Z__XZZ_', # node 8
        '_______ZZX__', # node 9
        '______Z_Z_X_', # node 10
        '______ZZ___X'  # node 11
        ]
        verify_stim_rgs_3arms_2bells(s) # check that s have correct queries

        # Operate measurement
        for index in c:
            # Update s, q
            s, q = measure_z_with_correction(index=index, s=s, queries=q)

        # Check if result match one of all possible results
        for result in all_stabilizer_result:
            if is_subset(result, q):
                # print(f"Found solution: {result}")   
                bell_found_list.append(result)                     
                found = True

        if found == True and len(bell_found_list) >= expected_num_bells:
            for bell in bell_found_list:
                print(f"{bell}")
            print(f'C: {c}')
            print('\n- - - - - - - - - - - - - - - - - - - - - - - -')

Measure: 1 nodes
Measure: 2 nodes
Measure: 3 nodes
Measure: 4 nodes
['_X___Z______', '_Z___X__Z___', '_____Z__X_Z_', '________Z_X_']
['__X_Z_______', '__Z_X__Z____', '____Z__X___Z', '_______Z___X']
C: (0, 3, 6, 9)

- - - - - - - - - - - - - - - - - - - - - - - -
['X____Z______', 'Z____X__Z___', '_____Z__XZ__', '________ZX__']
['__XZ________', '__ZX__Z_____', '___Z__X____Z', '______Z____X']
C: (1, 4, 7, 10)

- - - - - - - - - - - - - - - - - - - - - - - -
['X___Z_______', 'Z___X__Z____', '____Z__X_Z__', '_______Z_X__']
['_X_Z________', '_Z_X__Z_____', '___Z__X___Z_', '______Z___X_']
C: (2, 5, 8, 11)

- - - - - - - - - - - - - - - - - - - - - - - -
Measure: 5 nodes
Measure: 6 nodes
Measure: 7 nodes
Measure: 8 nodes
Measure: 9 nodes
Measure: 10 nodes


In [10]:
num_trial = 5
max_found_bells = 0

for i in tqdm(range(num_trial)):    

    for num_measurements in range(1, 11):
        # print(f'Measure: {num_measurements} nodes')
        combinations = generate_combinations_with_adjustable_replacement(
            nodes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 
            max_length=num_measurements, # You have to change this parameter, if you want to modify number of node operation
            max_self_combination=1
        )    

        for c in combinations:

            # Restart parameters before operate new measurement set
            found: bool = False
            bell_found_list = []

            q, circuit = generate_rgs_random(num_nodes=12, num_bell_between_row=2)
            s = stim.TableauSimulator()
            s.do_circuit(stim.Circuit(circuit))

            # Operate measurement
            for index in c:
                # Update s, q
                s, q = measure_z_with_correction(index=index, s=s, queries=q)

            # Check if result match one of all possible results
            for result in all_stabilizer_result:
                if is_subset(result, q):
                    # print(f"Found solution: {result}")   
                    bell_found_list.append(result)                     
                    found = True                       
            
            if len(bell_found_list) >= max_found_bells:
                max_found_bells = len(bell_found_list)

print(f'Max found bells: {max_found_bells}')

100%|██████████| 5/5 [00:23<00:00,  4.70s/it]

Max found bells: 2


In [18]:
combinations = generate_combinations_with_adjustable_replacement(
    nodes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 
    max_length=8, # You have to change this parameter, if you want to modify number of node operation
    max_self_combination=1
)

operation = ['-', 'Z']

for c in combinations:
    for op_1 in operation:
        for op_2 in operation:
            for op_3 in operation:
                for op_4 in operation:
                    for op_5 in operation:
                        for op_6 in operation:
                            for op_7 in operation:
                                for op_8 in operation:

                                    if c == (0, 1, 3, 5, 6, 8, 9, 10):
                                        if op_1 == 'Z' or op_2 == '-' or op_3 == 'Z' or op_4 == '-' or op_5 == 'Z' or op_6 == '-' or op_7 == 'Z' or op_8 == '-':
                                            print(c)
                                            print(op_1, op_2, op_3, op_4, op_5, op_6, op_7, op_8)
                                            print('Eu')

                                    # Restart parameters before operate new measurement set
                                    found: bool = False
                                    s = stim_rgs_3arms_2bells()
                                    q = [
                                        'X___ZZ______', # node 0
                                        '_X_Z_Z______', # node 1
                                        '__XZZ_______', # node 2
                                        '_ZZX__Z_____', # node 3
                                        'Z_Z_X__Z____', # node 4
                                        'ZZ___X__Z___', # node 5
                                        '___Z__X___ZZ', # node 6
                                        '____Z__X_Z_Z', # node 7
                                        '_____Z__XZZ_', # node 8
                                        '_______ZZX__', # node 9
                                        '______Z_Z_X_', # node 10
                                        '______ZZ___X'  # node 11
                                    ]
                                    verify_stim_rgs_3arms_2bells(s) # check that s have correct queries
                                    
                                    if op_1 == '-':
                                        continue
                                    elif op_1 == 'Z':
                                        s, q = measure_z_with_correction(index=c[0], s=s, queries=q)                            

                                    if op_2 == '-':
                                        continue
                                    elif op_2 == 'Z':
                                        s, q = measure_z_with_correction(index=c[1], s=s, queries=q)

                                    if op_3 == '-':
                                        continue
                                    elif op_3 == 'Z':
                                        s, q = measure_z_with_correction(index=c[2], s=s, queries=q)

                                    if op_4 == '-':
                                        continue
                                    elif op_4 == 'Z':
                                        s, q = measure_z_with_correction(index=c[3], s=s, queries=q)                

                                    if op_5 == '-':
                                        continue
                                    elif op_5 == 'Z':
                                        s, q = measure_z_with_correction(index=c[4], s=s, queries=q)

                                    if op_6 == '-':
                                        continue
                                    elif op_6 == 'Z':
                                        s, q = measure_z_with_correction(index=c[5], s=s, queries=q)

                                    if op_7 == '-':
                                        continue
                                    elif op_7 == 'Z':
                                        s, q = measure_z_with_correction(index=c[6], s=s, queries=q)

                                    if op_8 == '-':
                                        continue
                                    elif op_8 == 'Z':
                                        s, q = measure_z_with_correction(index=c[7], s=s, queries=q)                                    

                                    # Check if result match one of all possible results
                                    for result in all_stabilizer_result:
                                        if is_subset(result, q):
                                            print(f"Found solution: {result}")            
                                            found = True                                    

                                    if found == True:
                                        print(f'C: {c}')
                                        print(f'Ops: {op_1}, {op_2}, {op_3}, {op_4}, {op_5}, {op_6}, {op_7}, {op_8}')
                                        print('\n- - - - - - - - - - - - - - - - - - - - - - - -')


(0, 1, 3, 5, 6, 8, 9, 10)
- - - - - - - -
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - - - - - Z
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - - - - Z -
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - - - - Z Z
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - - - Z - -
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - - - Z - Z
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - - - Z Z -
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - - - Z Z Z
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - - Z - - -
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - - Z - - Z
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - - Z - Z -
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - - Z - Z Z
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - - Z Z - -
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - - Z Z - Z
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - - Z Z Z -
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - - Z Z Z Z
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - Z - - - -
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - Z - - - Z
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - Z - - Z -
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - Z - - Z Z
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - Z - Z - -
Eu
(0, 1, 3, 5, 6, 8, 9, 10)
- - - Z - Z - Z
Eu
(0, 1, 3, 

In [21]:
num_nodes = 16
[i for i in range(num_nodes)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]